In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [5]:
# Step 2: Create Binary Labels (Benign vs Attack)
df['Binary_Label'] = df['Label'].apply(lambda x: 'BENIGN' if x == 'BENIGN' else 'ATTACK')

# Confirm the label distribution
print(df['Binary_Label'].value_counts())


NameError: name 'df' is not defined

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Binary_Label_encoded'] = label_encoder.fit_transform(df['Binary_Label'])

# BENIGN → 0, ATTACK → 1
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


In [ ]:
# Drop unnecessary columns
columns_to_drop = ['Label', 'BroadLabel', 'Binary_Label']
feature_columns = df.drop(columns=columns_to_drop + ['Binary_Label_encoded']).select_dtypes(include=['number']).columns.tolist()

# Final feature set and label
X = df[feature_columns]
y = df['Binary_Label_encoded']

print("✅ Feature shape:", X.shape)


In [ ]:
from sklearn.model_selection import train_test_split

# Split: 60% train, 20% val, 20% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"✅ Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")


In [ ]:
from tensorflow import keras

model = keras.models.load_model('/Users/user/Downloads/best_model1.h5')


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Data scaled.")


In [ ]:
# Replace inf with NaN, then fill NaN with mean
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_val = X_val.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Fill missing values with column mean
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_train.mean())  # Use training mean for validation/test
X_test = X_test.fillna(X_train.mean())


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Data scaled.")


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use categorical_crossentropy if y_train is one-hot encoded
    metrics=['accuracy']
)


In [ ]:

x_train=X_train_scaled
x_val=X_val_scaled
x_test=X_test_scaled


history = model.fit(
    x_train, y_train,
    epochs=3,
    validation_split=0.2  # use 20% of training data as validation
)


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x_train, y_train,
    validation_split=0.2
)